In [2]:
from csv import writer
from csv import reader
def data_append(row, line_num):
    if line_num==1:
        return str(row.append('Delayed'))
    delay = row[8]
    dataApp=''
    if(delay==''):
        return str(row.append('0'))
    elif float(delay)>0.0:
        return str(row.append('1'))
    else:
        return str(row.append('0'))

def add_column_in_csv(input_file, output_file, transform_row):
    """ Append a column in existing csv using csv.reader / csv.writer classes"""
    # Open the input_file in read mode and output_file in write mode
    with open(input_file, 'r') as read_obj, \
            open(output_file, 'w', newline='') as write_obj:
        # Create a csv.reader object from the input file object
        csv_reader = reader(read_obj)
        # Create a csv.writer object from the output file object
        csv_writer = writer(write_obj)
        # Read each row of the input csv file as list
        for row in csv_reader:
            # Pass the list / row in the transform function to add column text for this row
            transform_row(row, csv_reader.line_num)
            # Write the updated row / list to the output file
            csv_writer.writerow(row)

In [6]:
add_column_in_csv('2009.csv', '2009_new.csv', lambda row, line_num: row.append('Delayed') if line_num==1 else (row.append('0') if row[7]=='' else (row.append('1') if(float(row[7])>0.0) else row.append('0'))))

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import StringType, IntegerType, DateType, DoubleType
from pyspark.sql import functions
import math
import plotly.offline as pyo
import plotly.graph_objs as go
from calendar import month
from pandas import to_datetime
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col
import pandas as pd

spark = SparkSession.builder.appName('ml').getOrCreate()
df = spark.read.csv('/home/ubuntu/data/ml/2009_new.csv', header = True, inferSchema = True)
df.printSchema()

/home/ubuntu/.local/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning:

Python 3.6 support is deprecated in Spark 3.2.



root
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_TIME: double (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- TAXI_OUT: double (nullable = true)
 |-- WHEELS_OFF: double (nullable = true)
 |-- WHEELS_ON: double (nullable = true)
 |-- TAXI_IN: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_TIME: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = true)
 |-- WEATHER_DELAY: do

In [2]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2009-01-01,2009-01-01,2009-01-01,2009-01-01,2009-01-01
OP_CARRIER,XE,XE,XE,XE,XE
OP_CARRIER_FL_NUM,1204,1206,1207,1208,1209
ORIGIN,DCA,EWR,EWR,DCA,IAD
DEST,EWR,IAD,DCA,EWR,EWR
CRS_DEP_TIME,1100,1510,1100,1240,1715
DEP_TIME,1058,1509,1059,1249,1705
DEP_DELAY,-2,-1,-1,9,-10
TAXI_OUT,18,28,20,10,24
WHEELS_OFF,1116,1537,1119,1259,1729


In [3]:
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2009-01-01,2009-01-01,2009-01-01,2009-01-01,2009-01-01
OP_CARRIER,XE,XE,XE,XE,XE
OP_CARRIER_FL_NUM,1204,1206,1207,1208,1209
ORIGIN,DCA,EWR,EWR,DCA,IAD
DEST,EWR,IAD,DCA,EWR,EWR
CRS_DEP_TIME,1100,1510,1100,1240,1715
DEP_TIME,1058,1509,1059,1249,1705
DEP_DELAY,-2,-1,-1,9,-10
TAXI_OUT,18,28,20,10,24
WHEELS_OFF,1116,1537,1119,1259,1729


In [4]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
OP_CARRIER_FL_NUM,6429338,2293.7116343859975,2046.497463763623,1,7829
CRS_DEP_TIME,6429338,1319.4466834688112,457.7529163543421,1,2359
CRS_ARR_TIME,6429338,1496.7219118049168,473.31775527788665,1,2400
Delayed,6429338,0.35071246837543774,0.4771931142831878,0,1


In [5]:
df = df.select('OP_CARRIER', 'DEP_DELAY', 'ARR_DELAY','Delayed')
cols = df.columns
df.printSchema()

root
 |-- OP_CARRIER: string (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- Delayed: integer (nullable = true)



In [6]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ['OP_CARRIER']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'Delayed', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['DEP_DELAY', 'ARR_DELAY']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features", handleInvalid="skip")
stages += [assembler]

In [7]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df = pipelineModel.transform(df)
selectedCols = ['label', 'features'] + cols
df = df.select(selectedCols)
df.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- Delayed: integer (nullable = true)



In [8]:
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
label,0,0,0,1,0
features,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ..."
OP_CARRIER,XE,XE,XE,XE,XE
DEP_DELAY,-2,-1,-1,9,-10
ARR_DELAY,4,-8,-9,-12,-38
Delayed,0,0,0,1,0


In [9]:
train, test = df.randomSplit([0.7, 0.3], seed = 2018)

train = df.filter(df.DEP_DELAY.isNotNull()).filter(df.ARR_DELAY.isNotNull()).filter(df.OP_CARRIER.isNotNull())

In [10]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [13]:
predictions = lrModel.transform(test)
predictions.select('OP_CARRIER', 'label', 'rawPrediction', 'prediction', 'probability').filter(predictions.prediction > 0).show(10)

+----------+-----+--------------------+----------+--------------------+
|OP_CARRIER|label|       rawPrediction|prediction|         probability|
+----------+-----+--------------------+----------+--------------------+
|        WN|  0.0|[-0.1860603350525...|       1.0|[0.45361864331531...|
|        WN|  0.0|[-0.2309341428542...|       1.0|[0.44252168351132...|
|        WN|  0.0|[-0.2533710467551...|       1.0|[0.43699394427345...|
|        AA|  0.0|[-0.0967179489827...|       1.0|[0.47583934377831...|
|        AA|  0.0|[-0.2986500840902...|       1.0|[0.42588751446765...|
|        AA|  0.0|[-0.0202694271969...|       1.0|[0.49493281668713...|
|        AA|  0.0|[-0.0202694271969...|       1.0|[0.49493281668713...|
|        AA|  0.0|[-0.0202694271969...|       1.0|[0.49493281668713...|
|        AA|  0.0|[-0.0427063310977...|       1.0|[0.48932503961963...|
|        AA|  0.0|[-0.0427063310977...|       1.0|[0.48932503961963...|
+----------+-----+--------------------+----------+--------------